In [9]:
%load_ext autoreload
%autoreload 2

import sys, os
import pandas as pd
import numpy as np
from tqdm import tqdm
import wandb

sys.path.append(os.path.join(os.getcwd(), '../symlie'))
from misc.utils import NumpyUtils, Results

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# results_df_old = pd.read_pickle('../logs/store/results_df.pkl')
results_df_old = pd.DataFrame(columns = ['run_id'])
results_df_old

,run_id


In [11]:
log_dir = '../logs'
store_dir = os.path.join(log_dir, 'store')
wandb_dir = os.path.join(log_dir, 'wandb')

run_dirs =  np.array([[run_dir.split('-')[-1], run_dir] for run_dir in os.listdir(wandb_dir) if run_dir.startswith('run')])
assert len(run_dirs[:, 0]) == len(set(run_dirs[:, 0]))
run_dirs = {run_id: run_dir for run_id, run_dir in run_dirs}
pd.DataFrame(run_dirs.items(), columns = ['run_id', 'run_dir'])

,run_id,run_dir
0,vh4jrs16,run-20240124_215834-vh4jrs16
1,lib7fb4f,run-20240125_132527-lib7fb4f


In [15]:
exclude_ids = ['1jjq8ar4']

In [18]:
api = wandb.Api()
runs = api.runs('eliasdubbeldam/symlie')
for run in runs:
    print(run.name, run.id)
    # if run.id in exclude_ids: run.delete()

distinctive-shadow-30 lib7fb4f
generous-wood-28 vh4jrs16


In [20]:
config_list = []

pbar = tqdm(runs)
for run in pbar:
    pbar.set_description(f'Retreiving wandb {str(id)}')
    
    id = run.id

    # Exceptions
    if run.state != 'finished':
        continue
    if id in results_df_old['run_id'].values:
        continue
    if id in exclude_ids:
        continue


    # Retreive config from wandb run, add all info to config
    config = run.config

    # Add run id and name to config
    config['run_id'] = id
    config['run_name'] = run.name

    # Retreive metrics from wandb run
    history = run.history()
    metrics = ['train_loss', 'val_loss', 'test_loss']
    for metric in metrics:
        config[metric] = np.array(history[metric].dropna())

    # Retreive arrays from wandb files
    npu = NumpyUtils(dir = os.path.join(wandb_dir, run_dirs[id], 'files'))
    run_arrays = npu.load_all()
    for filename, array in run_arrays.items():
        config[filename] = array
    

    config_list.append(config)

results_df = pd.DataFrame(config_list)

results_df_new = pd.concat([results_df_old, results_df])
results_df_new.to_pickle(os.path.join(store_dir, 'results_df.pkl'))

Retreiving wandb lib7fb4f: 100%|██████████| 2/2 [00:00<00:00,  2.70it/s]


In [22]:
results_df_new

,run_id,lr,net,bias,name,seed,test,train,y_low,device,...,persistent_workers,run_name,train_loss,val_loss,test_loss,out_a_prime,P,out_b_prime,y_true,y_pred
0,lib7fb4f,0.100,TrainP,False,None,42.0,True,True,1.0,cpu,...,True,distinctive-shadow-30,"[0.1514815241098404, 0.14383874833583832, 0.13...","[0.15973155200481415, 0.13493697345256805, 0.0...",[0.1257612109184265],"[[0.24581842, -0.09962792, 0.059194632, -0.059...","[[4.10627, 3.0690331, 3.4362614, -4.6359572, 2...","[[0.19467984, -0.16883406, -0.10459621, -0.327...",NaN,NaN
1,vh4jrs16,0.001,Predict-NoneP,False,None,1.0,True,True,1.0,NaN,...,True,generous-wood-28,[0.37597087025642395],[0.2678754925727844],[0.27310794591903687],NaN,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",NaN,"[2.0, 1.0, 2.0, 1.0, 2.0, 2.0, 1.0, 1.0, 2.0, ...","[1.4355762, 1.4824753, 1.3475084, 1.4821842, 1..."


In [21]:
import matplotlib.pyplot as plt

In [ ]:
loss = np.stack(results_df_new['val_loss'])
plt.plot( loss.T )
plt.show()

loss